### Import Dependencies

In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

#### Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

#### Create Database Connection

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

####  Reflect an existing database into a new model

In [6]:
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

#### We can view all of the classes that automap found

In [7]:
Base.classes.keys()

['measurement', 'station']

#### Save references to each table

In [8]:
Measurement = Base.classes.measurement
Station = Base.classes.station

#### Create our session (link) from Python to the DB

In [9]:
session = Session(engine)

#### Getting column names for measurement table

In [12]:
inspector = inspect(engine)
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


#### Getting column names for Station table

In [13]:
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


### Exploratory Climate Analysis

### Precipitation Analysis

##### Query last date in dataframe

In [14]:
last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
last_date

('2017-08-23')

##### Calculate the date 1 year ago from the last data point in the database

In [15]:
one_year_ago = dt.date(2017,8,23) - dt.timedelta(days=365)
one_year_ago

datetime.date(2016, 8, 23)

##### Perform a query to retrieve the data and precipitation scores

In [23]:
all_scores = session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.desc()).limit(10).all()
all_scores

[('2017-08-23', 0.0),
 ('2017-08-23', 0.0),
 ('2017-08-23', 0.08),
 ('2017-08-23', 0.45),
 ('2017-08-22', 0.0),
 ('2017-08-22', 0.0),
 ('2017-08-22', 0.5),
 ('2017-08-21', 0.0),
 ('2017-08-21', 0.02),
 ('2017-08-21', None)]

#####  Query to retrieve the Last 12 Months of Precipitation Data Selecting Only the `date` and `prcp` Values

In [28]:
prcp_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).order_by(Measurement.date).limit(10).all()
prcp_data

[('2016-08-23', 0.0),
 ('2016-08-23', 0.15),
 ('2016-08-23', 0.05),
 ('2016-08-23', None),
 ('2016-08-23', 0.02),
 ('2016-08-23', 1.79),
 ('2016-08-23', 0.7),
 ('2016-08-24', 0.08),
 ('2016-08-24', 2.15),
 ('2016-08-24', 2.28)]

##### Save the query results as a Pandas DataFrame and set the index to the date column

In [31]:
prcp_df = pd.DataFrame(prcp_data, columns=["Date","Precipit"])
prcp_df.set_index("Date", inplace=True,)
prcp_df.head(10)

,Precipit
Date,
2016-08-23,0.00
2016-08-23,0.15
2016-08-23,0.05
2016-08-23,NaN
2016-08-23,0.02
2016-08-23,1.79
2016-08-23,0.70
2016-08-24,0.08
2016-08-24,2.15
